In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset,TensorDataset
from sklearn.model_selection import KFold
from learning_tool import *
import sys
sys.path.append('..')

from model.model import *

In [3]:
!nvidia-smi

Tue Aug  1 16:36:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB           On | 00000000:07:00.0 Off |                    0 |
| N/A   69C    P0              222W / 400W|  28298MiB / 81920MiB |     93%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from tqdm.notebook import tqdm
from IPython.display import display
from sklearn.model_selection import train_test_split

In [3]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [4]:
# デバイスの割り当て

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# device = torch.device('cpu')

cuda:0


In [5]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df, features, labels):
        self.features_values = df[features].values
        self.labels = df[labels].values
        
    # len()を使用すると呼ばれる
    def __len__(self):
        return len(self.features_values)

    # 要素を参照すると呼ばれる関数    
    def __getitem__(self, idx):
        features_x = torch.LongTensor(self.features_values[idx])
        labels = torch.as_tensor(self.labels[idx])
        return features_x, labels

In [6]:
n_output = 2
n_hidden = 100
print(f'n_hidden: {n_hidden} n_output: {n_output}')

n_hidden: 100 n_output: 2


In [7]:
# PyTorch乱数固定用

def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True
    
# # 乱数初期化
# torch_seed()

In [8]:
def undersampling(dataset):
    target = np.zeros(0)
    target = (np.hstack(data[1].numpy() for data in dataset))
    print (
        'original target  0/1: {}/{}'.format(
        len(np.where(target == 0)[0]), len(np.where(target == 1)[0]))
    )
    
    class_indices = [np.where(target == t)[0] for t in np.unique(target)]
    class_sample_count = np.array([len(idx) for idx in class_indices])
    min_class_idx = np.argmin(class_sample_count)
    max_class_idx = np.argmax(class_sample_count)
    undersample_indices = np.random.choice(class_indices[max_class_idx], size=int(min(class_sample_count)), replace=False)
    combined_indices = np.hstack([class_indices[min_class_idx], undersample_indices])
    
    features = dataset[combined_indices][0]
    labels = dataset[combined_indices][1]
    
    shuffle_indices = torch.randperm(len(features))
    features = features[shuffle_indices]
    labels = labels[shuffle_indices]
    
    undersampled_dataset = torch.utils.data.TensorDataset(features, labels)
    
    target = np.zeros(0)
    target = (np.hstack(data[1].numpy() for data in undersampled_dataset))
    print (
        'undersampled target  0/1: {}/{}'.format(
        len(np.where(target == 0)[0]), len(np.where(target == 1)[0]))
    )
    return undersampled_dataset

In [9]:
def make_dataset(filename):
    df = pd.read_csv(filename, index_col=0)
    columns = df.columns.values
    features_columns = columns[:-1]
    labels_column = columns[-1]
    dataset = MyDataset(df, features_columns, labels_column)
    return dataset

In [10]:
def record_history(df, filename, seed, each_history, num_splits):
    # print(df)
    score_type = ['train_loss','train_acc','test_loss','test_acc','auc_score']
    for num_kf in range(num_splits):
        for i in range(5):
            tmp_list = list(each_history[num_kf,:,i+1])
            tmp_list[:0] = [filename, seed, num_kf+1, score_type[i]]
            # print(tmp_list)
            df.loc[len(df)] = tmp_list
    # print(df)

In [11]:
# 学習率
lr = 0.001

# 繰り返し回数
num_epochs = 35

batch_size = 66
num_splits = 10
kf = KFold(n_splits=num_splits)

columns_list = [i for i in range(1,num_epochs+1)]
columns_list[:0] = ['filename','seed', 'num_kf', 'score_type']
columns_list
df = pd.DataFrame(columns=columns_list)

for i in range(5,20):
    filename = 'txt_vec_and_label_camel_'+str(i)+'.csv'
    copy_df = df.copy()
    path = '../resource/' + filename
    dataset = make_dataset(path)
    print(filename)
    
    # random_seed_list = [100,101,102,103,104,105,106,107,108,109,110]
    random_seed_list = [100]
    for seed in random_seed_list:
        print(seed)
        cv_cnt = 0
        torch_seed(seed)
        undersampled_dataset = undersampling(dataset)
        # cv評価結果記録用
        each_history = np.zeros((0,num_epochs,6))

        for train_index, test_index in kf.split(undersampled_dataset):
            cv_cnt += 1
            print(f'cv: {cv_cnt}')
            history = np.zeros((0,6))
            train_dataset = Subset(undersampled_dataset, train_index)
            train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2)
            test_dataset   = Subset(undersampled_dataset, test_index)
            test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=2)

            # モデルインスタンス生成
            net = CNN(n_output, n_hidden).to(device)
            # net._initialize_weights()
            # 最適化関数: 勾配降下法
            optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.001)
            # optimizer = torch.optim.SGD(net.parameters(), lr=lr)
            # 損失関数： 交差エントロピー関数
            criterion = nn.CrossEntropyLoss()

            #学習
            history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, test_dataset)

            #1交差ごとの記録
            each_history = np.vstack((each_history, [history]))

        record_history(copy_df, filename, seed, each_history, num_splits)
        print(copy_df)
    copy_df.to_csv('../result/threshold_2000_epoch_35_camel_'+str(i)+'.csv', mode='w',index=False)

txt_vec_and_label_camel_5.csv
100
original target  0/1: 16335/9582
undersampled target  0/1: 9582/9582
cv: 1


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch [1/35], loss: 0.67155 acc: 0.65374 val_loss: 0.66592, val_acc: 0.63015, auc_score: 0.76240


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch [2/35], loss: 0.59531 acc: 0.68505 val_loss: 0.57012, val_acc: 0.70892, auc_score: 0.77075


...

Epoch [35/35], loss: 0.47176 acc: 0.77864 val_loss: 0.58255, val_acc: 0.69468, auc_score: 0.77182
                          filename  seed  num_kf  score_type         1   
0   txt_vec_and_label_camel_12.csv   100       1  train_loss  0.679587  \
1   txt_vec_and_label_camel_12.csv   100       1   train_acc  0.644460   
2   txt_vec_and_label_camel_12.csv   100       1   test_loss  0.642499   
3   txt_vec_and_label_camel_12.csv   100       1    test_acc  0.646322   
4   txt_vec_and_label_camel_12.csv   100       1   auc_score  0.760489   
5   txt_vec_and_label_camel_12.csv   100       2  train_loss  0.676960   
6   txt_vec_and_label_camel_12.csv   100       2   train_acc  0.647417   
7   txt_vec_and_label_camel_12.csv   100       2   test_loss  0.573517   
8   txt_vec_and_label_camel_12.csv   100       2    test_acc  0.705790   
9   txt_vec_and_label_camel_12.csv   100       2   auc_score  0.762829   
10  txt_vec_and_label_camel_12.csv   100       3  train_loss  0.676804   
11  txt_vec_an

  0%|          | 0/262 [00:00<?, ?it/s]

Epoch [1/35], loss: 0.67591 acc: 0.65026 val_loss: 0.68130, val_acc: 0.61033, auc_score: 0.74950


  0%|          | 0/262 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# evaluate_history(cv_history)

In [ ]:
# evaluate_history(each_history[1,:,:])

In [ ]:
# each_history[0,:,5]

In [ ]:
# df.to_csv('../result/threshold_2000_camle_0.csv', index=False)

In [ ]:
# for i in range(0, 75, 5):
#     print(i)

In [ ]:
print(each_history)

In [ ]:
copy_df